# GODBRAIN Demo Notebook

This notebook demonstrates how to use the GODBRAIN system for interactive reasoning and question answering. GODBRAIN combines symbolic reasoning with a knowledge graph and metacognitive abilities to provide answers with explanations and confidence scores.

## Components of GODBRAIN:

1. **Perception Layer** (TextParser): Processes natural language input
2. **Logical Core** (ReasoningEngine): Performs symbolic reasoning using Prolog
3. **World Model** (KnowledgeGraph): Maintains a graph-based representation of knowledge
4. **Meta-Cognition Unit** (ConfidenceEvaluator): Evaluates confidence in answers and can revise them


## Setup and Initialization

First, let's import the necessary modules and initialize the GODBRAIN system.

In [ ]:
import sys
import os

# Add the parent directory to the path so we can import the GODBRAIN modules
sys.path.append(os.path.abspath('../'))

# Import the GODBRAIN system
from src.godbrain import GODBRAIN

# Initialize the GODBRAIN system with a confidence threshold of 0.7
brain = GODBRAIN(confidence_threshold=0.7)

print("GODBRAIN system initialized successfully!")


## Exploring the Knowledge Base

The GODBRAIN system is initialized with some basic knowledge about philosophers. Let's examine what it knows.

In [ ]:
# Display the facts and rules in the logical core
print("Facts and rules in the logical core:
")
for item in brain.logical_core.knowledge_base:
    print(f"  {item}")

print("
Entities in the world model:
")
for entity in brain.world_model.get_all_entities():
    print(f"  {entity}: {brain.world_model.entity_attributes.get(entity, {})}")

print("
Relationships in the world model:
")
for subject, predicate, object_ in brain.world_model.get_all_relationships():
    print(f"  {subject} {predicate} {object_}")


## Processing a Simple Query

Let's try processing a simple query to see how GODBRAIN works.

In [ ]:
# Process a simple query
query = "Is Socrates mortal?"
result = brain.process_input(query)

print(f"Query: {query}")
print(f"Answer: {result['answer']}")
print(f"Confidence: {result['confidence']:.2f}")
print("Explanation:
{}".format(result['explanation']))


## Adding New Knowledge

We can extend GODBRAIN's knowledge by adding new facts and relationships.

In [ ]:
# Add new facts to the logical core
brain.logical_core.add_fact("human(pythagoras).")
brain.logical_core.add_fact("teacher(pythagoras, empedocles).")
brain.logical_core.add_fact("human(empedocles).")

# Add corresponding entities and relationships to the world model
brain.world_model.add_entity('pythagoras', {'type': 'person'})
brain.world_model.add_entity('empedocles', {'type': 'person'})

brain.world_model.add_relationship('pythagoras', 'is_a', 'human')
brain.world_model.add_relationship('empedocles', 'is_a', 'human')
brain.world_model.add_relationship('pythagoras', 'teacher_of', 'empedocles')
brain.world_model.add_relationship('pythagoras', 'is_a', 'philosopher')

print("New knowledge added successfully!")


## Testing with the New Knowledge

Let's test GODBRAIN with queries about the new knowledge we just added.

In [ ]:
# Test queries with the new knowledge
queries = [
    "Is Pythagoras mortal?",
    "Is Pythagoras a philosopher?",
    "Who is the teacher of Empedocles?"
]

for query in queries:
    result = brain.process_input(query)
    print(f"
Query: {query}")
    print(f"Answer: {result['answer']}")
    print(f"Confidence: {result['confidence']:.2f}")
    print("Explanation:
{}".format(result['explanation']))


## Interactive Chat Interface

Now let's create an interactive chat interface to interact with GODBRAIN.

In [ ]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Create the chat history and input widgets
chat_history = widgets.HTML(
    value='<div style="height: 400px; overflow-y: auto; background-color: #f5f5f5; padding: 10px; border-radius: 5px;"></div>',
    layout=widgets.Layout(width='100%', height='400px')
)

user_input = widgets.Text(
    placeholder='Ask GODBRAIN a question...',
    layout=widgets.Layout(width='80%')
)

send_button = widgets.Button(
    description='Send',
    button_style='primary',
    layout=widgets.Layout(width='19%')
)

input_box = widgets.HBox([user_input, send_button])

# Function to update the chat history
def update_chat(sender=None):
    query = user_input.value.strip()
    if not query:
        return
    
    # Clear the input box
    user_input.value = ''
    
    # Get the current chat history
    current_history = chat_history.value
    
    # Add the user's query to the chat history
    current_history = current_history.replace('</div>', f'<p><strong>You:</strong> {query}</p></div>')
    chat_history.value = current_history
    
    # Process the query with GODBRAIN
    result = brain.process_input(query)
    
    # Format the response
    answer = result['answer']
    confidence = result['confidence']
    explanation = result['explanation'].replace('
', '<br>')
    
    # Add the response to the chat history
    confidence_color = 'green' if confidence >= 0.7 else 'orange' if confidence >= 0.4 else 'red'
    response_html = f'''
    <p><strong>GODBRAIN:</strong> {answer}</p>
    <p><small style="color: {confidence_color};">Confidence: {confidence:.2f}</small></p>
    <p><small><em>Explanation:</em><br>{explanation}</small></p>
    <hr>
    '''
    
    current_history = chat_history.value
    current_history = current_history.replace('</div>', f'{response_html}</div>')
    chat_history.value = current_history

# Connect the button and input box to the update_chat function
send_button.on_click(update_chat)
user_input.on_submit(update_chat)

# Display the chat interface
display(widgets.VBox([
    widgets.HTML('<h3>Chat with GODBRAIN</h3>'),
    chat_history,
    input_box
]))


## Example Queries

Here are some example queries you can try with GODBRAIN:

1. Is Socrates mortal?
2. Is Plato a philosopher?
3. Who is the teacher of Plato?
4. Is Empedocles a student of Pythagoras?
5. What is a human?

Feel free to try your own queries as well!

## Visualizing the Knowledge Graph

Let's visualize the knowledge graph to see the relationships between entities.

In [ ]:
# Visualize the knowledge graph
import matplotlib.pyplot as plt
import networkx as nx

# Get the graph from the world model
G = brain.world_model.graph

# Create a figure with a larger size
plt.figure(figsize=(12, 8))

# Define node colors based on entity type
node_colors = []
for node in G.nodes():
    entity_type = brain.world_model.entity_attributes.get(node, {}).get('type', 'unknown')
    if entity_type == 'person':
        node_colors.append('lightblue')
    elif entity_type == 'class':
        node_colors.append('lightgreen')
    else:
        node_colors.append('lightgray')

# Draw the graph
pos = nx.spring_layout(G, seed=42)  # Seed for reproducible layout
nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=2000, font_size=10, font_weight='bold')

# Draw edge labels
edge_labels = {}
for u, v, data in G.edges(data=True):
    edge_labels[(u, v)] = data.get('predicate', '')

nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

plt.title('GODBRAIN Knowledge Graph')
plt.axis('off')  # Turn off the axis
plt.tight_layout()
plt.show()


## Conclusion

In this notebook, we've demonstrated how to use the GODBRAIN system for interactive reasoning and question answering. We've seen how the system combines symbolic reasoning with a knowledge graph and metacognitive abilities to provide answers with explanations and confidence scores.

The GODBRAIN architecture consists of four main components:

1. **Perception Layer**: Processes natural language input
2. **Logical Core**: Performs symbolic reasoning using Prolog
3. **World Model**: Maintains a graph-based representation of knowledge
4. **Meta-Cognition Unit**: Evaluates confidence in answers and can revise them

These components work together to create a system that can reason about the world and answer questions with explanations and confidence scores.